Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [2]:
df['gender'].value_counts(normalize=True)

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

##### 2. Какое количество уникальных значений у поля InternetService?

In [3]:
df['InternetService'].nunique()

3

In [4]:
df['InternetService'].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].describe()

count    7032.000000
mean     2283.300441
std      2266.771362
min        18.800000
25%       401.450000
50%      1397.475000
75%      3794.737500
max      8684.800000
Name: TotalCharges, dtype: float64

In [7]:
df['TotalCharges'].median()

1397.475

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [8]:
df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = 1
df.loc[df['PhoneService'] == 'No', 'PhoneService'] = 0
df['PhoneService'].head()

0    0
1    1
2    1
3    0
4    1
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

pd.to_numeric в пункте 4 уже замнил пробелы на NaN

In [9]:
df['TotalCharges'] = df['TotalCharges'].astype('float32').fillna(0)
df['TotalCharges'].isna().sum()

0

In [10]:
df['TotalCharges'].describe()

count    7043.000000
mean     2279.734375
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

In [11]:
df['TotalCharges'].median()

1394.55

Видим, что статистики немного сместились.

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [12]:
df.loc[df['Churn'] == 'Yes', 'Churn'] = 1
df.loc[df['Churn'] == 'No', 'Churn'] = 0
df['Churn'].head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [13]:
df.loc[df['StreamingMovies'] == 'Yes', 'StreamingMovies'] = 1
df.loc[(df['StreamingMovies'] == 'No') | (df['StreamingMovies'] == 'No internet service'), 'StreamingMovies'] = 0
df.loc[df['StreamingTV'] == 'Yes', 'StreamingTV'] = 1
df.loc[(df['StreamingTV'] == 'No') | (df['StreamingTV'] == 'No internet service'), 'StreamingTV'] = 0
df.loc[df['TechSupport'] == 'Yes', 'TechSupport'] = 1
df.loc[(df['TechSupport'] == 'No') | (df['TechSupport'] == 'No internet service'), 'TechSupport'] = 0
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head()

,StreamingMovies,StreamingTV,TechSupport
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,1
4,0,0,0


##### 8. Заполните пропуски в поле PhoneService значением 0

In [14]:
df['PhoneService'].unique()

array([0, 1], dtype=int64)

Не нашёл пропусков.

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [15]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [16]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], random_state=42)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

estimators = [('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))]
gender = Pipeline(estimators)

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [18]:
gender.fit_transform(X_train, y_train)

,gender_Female,gender_Male
6607,0,1
2598,1,0
2345,1,0
4093,1,0
693,1,0
...,...,...
3772,0,1
5191,1,0
5226,0,1
5390,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [19]:
from sklearn.preprocessing import StandardScaler

tenure = Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

In [20]:
tenure.fit_transform(X_train, y_train)

array([[-1.28288214],
       [-1.03785653],
       [-1.16036933],
       ...,
       [-0.83366851],
       [-0.83366851],
       [-0.26194207]])

Ответ - полезно, если для предсказаний используем модель, использующую расстояния. Например: линейная или логистическая регрессия, KNN, SVM и др.

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [21]:
#Ваш код здесь
TotalCharges = Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

In [22]:
TotalCharges.fit_transform(X_train, y_train)

array([[-0.9970656 ],
       [-0.77679443],
       [-0.97052896],
       ...,
       [-0.873309  ],
       [-0.47918373],
       [-0.80663705]], dtype=float32)

Объединение всех "кубиков" очень легко сделать таким образом

In [23]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [24]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
#feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tenure',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('TotalCharges',
                                                 Pipeline(memory=None,
                        

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [26]:
preds = pipeline.predict_proba(X_test)
preds[:10]

array([[0.27666667, 0.72333333],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.02      , 0.98      ],
       [0.88      , 0.12      ],
       [0.87      , 0.13      ],
       [0.9       , 0.1       ],
       [1.        , 0.        ],
       [0.56      , 0.44      ],
       [0.50428571, 0.49571429]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [27]:
from sklearn.metrics import roc_auc_score, log_loss

y_pred_proba = preds[:, 1]
print(f'roc_auc_score: {roc_auc_score(y_test, y_pred_proba)}')
print(f'log_loss: {log_loss(y_test, y_pred_proba)}')

roc_auc_score: 0.801335498096333
log_loss: 0.6715687228498308


### Сохраним наш пайплайн

In [28]:
#!pip install dill

In [29]:
#!pip show dill

In [30]:
#!python --version

In [31]:
# Установил модуль dill, но при импорте выдает ошибку: ModuleNotFoundError: No module named 'dill'
#import dill
#with open("model_RF.dill", "wb") as f:
#    dill.dump(pipeline, f)
    
import pickle
with open("model_RF.pkl", "wb") as f:
    pickle.dump(pipeline, f)

### Как ещё можно поиграться с пайплайном (Pipeline + GridSearch)

Подробный гайд тут - https://scikit-learn.org/stable/modules/compose.html#pipeline<br>
Примеры тут - https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#examples-using-sklearn-pipeline-pipeline<br>
Про подбор параметров - https://scikit-learn.org/stable/modules/grid_search.html#grid-search

Удобство совмещения Pipeline и GridSearch в том, что можно кросс-валидировать также и различные трансформаторы. Например, ниже в примере подбираем компоненты в PCA.

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC

estimators = [('features', feats), ('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)

# Individual steps may also be replaced as parameters, and non-final steps may be ignored by setting them to 'passthrough'
param_grid = dict(reduce_dim__n_components=[3, 5, 7],
                   clf=[SVC(), LogisticRegression()],
                   clf__C=[0.1, 1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring='roc_auc', cv=5)
grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('tenure',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
                                                                                         NumberSelector(key='tenure')),
                                                                                        ('standard',
                                                                                         StandardScaler(copy=True,
                                                                                                        with_mean=True,
                                                                    

In [33]:
grid_search.best_score_

0.7912449313956507

In [34]:
grid_search.best_params_

{'clf': LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'clf__C': 10,
 'reduce_dim__n_components': 7}